In [1]:
from pathlib import Path
import re

from dataclasses import dataclass
from graphlib import TopologicalSorter

In [2]:
init, ops_str = Path("data/24.txt").read_text().strip().split("\n\n")
l = {(s := x.split(": "))[0]: bool(int(s[1])) for x in init.split("\n")}


@dataclass(eq=True, frozen=True)
class OP:
    a: str
    b: str
    r: str
    op: str


pattern = re.compile("([a-z0-9]+) (AND|XOR|OR) ([a-z0-9]+) -> ([a-z0-9]+)")
pops = {"XOR": "^", "OR": "|", "AND": "&"}


def p(s: str):
    m = pattern.match(s)
    return OP(a=m.group(1), b=m.group(3), r=m.group(4), op=pops[m.group(2)])


ops = {(op := p(op_s)).r: op for op_s in ops_str.split("\n")}

In [3]:
graph = {op: {ops[x] for x in [op.a, op.b] if x in ops} for op in ops.values()}
ts = TopologicalSorter(graph)
for op in ts.static_order():
    l[op.r] = eval(f"{op.a} {op.op} {op.b}", l, {})

int(
    "".join(
        [str(int(v)) for k, v in sorted(l.items(), reverse=True) if k.startswith("z")]
    ),
    2,
)

52956035802096